In [1]:
# Monta o Google Drive para acessar arquivos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Instalação das bibliotecas necessárias
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install triton
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-pqyywd6g/unsloth_1ae73e49e12d40f1b842dded460e3696
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-pqyywd6g/unsloth_1ae73e49e12d40f1b842dded460e3696
  Resolved https://github.com/unslothai/unsloth.git to commit e80d642bc777f7a219bdd34aea1a77751f066785
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.9/126.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 13.3 MB/s eta 0:00:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [5]:
# Configurações básicas do modelo
max_seq_length = 512
dtype = None
load_in_4bit = True

In [4]:
# Função que formata o dataset com os campos instruction, input e output
def format_dataset_into_model_input(data):
    instructions, inputs, outputs = [], [], []

    for example in data:
        title = example['title']
        content = example['content']
        instruction = "Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers."
        instructions.append(instruction)
        inputs.append(title)
        outputs.append(content)

    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salva o dataset formatado
    formatted_json_path = "/content/drive/MyDrive/Notebooks/formatted_amazon_product_data.json"
    with open(formatted_json_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

    print(f"Dataset formatado salvo em: {formatted_json_path}")
    return formatted_json_path

In [6]:
# Caminho do dataset original
DATA_PATH = '/content/drive/MyDrive/Notebooks/trn.json'

# Carrega o dataset com o HuggingFace Datasets
dataset = load_dataset("json", data_files=DATA_PATH)

# Limita o dataset a 1000 exemplos e formata
raw_data = dataset['train'].select(range(1000))
formatted_path = format_dataset_into_model_input(raw_data)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset formatado salvo em: /content/drive/MyDrive/Notebooks/formatted_amazon_product_data.json


In [7]:
# Carrega o modelo pré-treinado (Mistral 7B, em 4-bit)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2025.3.18: Fast Mistral patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [8]:
# Prepara o modelo para inferência
FastLanguageModel.for_inference(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): M

In [9]:
# Função de geração de descrição com base no título
def generate_description(title):
    input_text = f"Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.\nTítulo: {title}\nDescrição:"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return description

# Teste de geração
print(generate_description("Smartphone Samsung Android 64GB"))
print(generate_description("Smartphone iPhone 64GB"))

Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone Samsung Android 64GB
Descrição:

The Samsung Galaxy S20 FE is a powerful smartphone that offers a range of features and capabilities to enhance your mobile experience. With its 6.5-inch Super AMOLED display, you can enjoy vibrant and clear visuals for your favorite movies, games, and more. The phone is powered by a Qualcomm Snapdragon 865 processor, ensuring smooth and efficient performance.

The Galaxy S20 FE features a triple-lens camera system, including a 12MP ultra-wide lens, a 12MP wide-angle lens, and an 8MP
Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Smartphone iPhone 64GB
Descrição:

The iPhone 64GB is a powerful and versatile smartphone that offers a range of

In [10]:
# Aplica LoRA (Low-Rank Adaptation) para treinar o modelo de forma leve
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Template no estilo Alpaca para transformar os dados em instruções
alpaca_prompt = """Below is an instruction outlining the task, accompanied by a title that provides additional context. Write a response that appropriately concludes the description.

### Instrução:
{}

### Título:
{}

### Descrição:
{}"""

EOS_TOKEN = tokenizer.eos_token

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.18 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [11]:
# Função que aplica o template nos dados
def formatting_prompts_func(data):
    texts = []
    for instruction, input_text, output in zip(data["instruction"], data["input"], data["output"]):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Carrega novamente o dataset formatado
dataset = load_dataset("json", data_files=formatted_path)

# Aplica o template nos dados
formatted_dataset = dataset['train'].map(formatting_prompts_func, batched=True)

# Configura e treina o modelo com os dados formatados
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=60,
        learning_rate=1e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
# Inicia o treinamento
trainer.train()

# Salva o modelo e tokenizer
model.save_pretrained("/content/drive/MyDrive/Notebooks/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Notebooks/lora_model")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 83,886,080/7,000,000,000 (1.20% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.445000
20,1.998800
30,1.294400
40,1.339100
50,1.226800
60,1.314500


('/content/drive/MyDrive/Notebooks/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Notebooks/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Notebooks/lora_model/tokenizer.model',
 '/content/drive/MyDrive/Notebooks/lora_model/added_tokens.json',
 '/content/drive/MyDrive/Notebooks/lora_model/tokenizer.json')

In [13]:
# Carrega novamente o modelo treinado para uso
model, tokenizer = FastLanguageModel.from_pretrained("/content/drive/MyDrive/Notebooks/lora_model")
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.3.18: Fast Mistral patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/Notebooks/lora_model as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
    

In [16]:
# Testes de inferência com o modelo ajustado
print(generate_description("Walking on Air"))

Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Walking on Air
Descrição:

'A beautiful, funny, heartbreaking book. I loved it.' - Cathy Rentzenbrink, author of The Last Act of Love

'A beautiful, funny, heartbreaking book. I loved it.' - Cathy Rentzenbrink, author of The Last Act of Love

'A beautiful, funny, heartbreaking book. I loved it.' - Cathy Rentzenbrink, author of The Last Act of Love

'A beautiful, funny, heartbreaking book. I loved it.' - Cathy Rentzenbrink, author of The Last Act of Love


In [17]:
print(generate_description("Mog and Bunny"))

Provide a detailed and engaging product description with a unique, creative style that highlights its key features and benefits, making it appealing to potential buyers.
Título: Mog and Bunny
Descrição:

### Sobre o Autor

Judith Kerr was born in Berlin. Her family left Germany in 1933, and came to England. She studied at the Central School of Art and later worked as a script-writer for the BBC. Judith Kerr has written over twenty books for children. The Tiger Who Came to Tea first published in 1968 has never been out of print in this country and has the most incredible history. It has been published in 20 languages and sold over five million copies. When it was first published it remained on the bests
